This program transfer the raw tiff file from scanimage to useful and convenient format  <br /> 
If the tiff file is volumetric, different slices will be separated and saved into different files. The last number of the file is the number of slices. 


In [1]:
from ScanImageTiffReader import ScanImageTiffReader
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import imageio
import os
import caiman as cm
from os import listdir
from os.path import isfile, join
import json

In [2]:
#The function can find specific parameter in metadata
def find_tag(metadata,name):
    string = metadata.find(name)
    temp_str = metadata[string:string+40]
    end = temp_str.find('\n')
    start = len(name)+3
    if name == 'scanFrameRate':
        value = float(temp_str[start:end])
    else:
        value = int(temp_str[start:end])
    return value

In [3]:
path = '/media/david/hdd1/David_GCAMP/20210812'#Choose the path that store your tif
os.chdir(path)
cwd = os.getcwd()
if not os.path.isdir(cwd+'/tag'):
    os.mkdir(cwd+'/tag')
if not os.path.isdir(cwd+'/sliced_data'):
    os.mkdir(cwd+'/sliced_data')
files = [f for f in listdir(path+'/raw') if isfile(join(path+'/raw', f))]

In [5]:
for filename in files:
    #Read tiff file
    vol=ScanImageTiffReader(cwd+'/raw/'+filename).data()
    metadata =ScanImageTiffReader(cwd+'/raw/'+filename).metadata()
    #Read tag
    total_volume = find_tag(metadata,'actualNumVolumes')
    framesPerSlice = find_tag(metadata,'framesPerSlice')
    num_slice = find_tag(metadata,'actualNumSlices')
    FrameRate = find_tag(metadata,'scanFrameRate')
    numFramesPerVolume = find_tag(metadata,'numFramesPerVolume')
    pixelsPerLine = find_tag(metadata,'pixelsPerLine')
    linesPerFrame = find_tag(metadata,'linesPerFrame')
    scanZoomFactor = find_tag(metadata,'scanZoomFactor')
    stackZStepSize = find_tag(metadata,'stackZStepSize')
    length = framesPerSlice*total_volume#time step
    #Store tag
    tag = {'total_volume':total_volume,'framesPerSlice':framesPerSlice,'framesPerSlice':framesPerSlice,'num_slice':num_slice,'FrameRate':FrameRate,'numFramesPerVolume':numFramesPerVolume,'pixelsPerLine':pixelsPerLine,'linesPerFrame':linesPerFrame,'scanZoomFactor':scanZoomFactor,'stackZStepSize':stackZStepSize,'metadata':metadata,'length':length}
    with open(cwd+'/tag/'+filename[0:-4]+'.json', 'w') as f:
        json.dump(tag, f)
    
    real_data = np.zeros((framesPerSlice*total_volume,vol.shape[1],vol.shape[2],num_slice))
    #Reshape the data (Seperate different layers)
    for slice in range(num_slice):
        for volume in range(total_volume):
            real_data[framesPerSlice*volume:framesPerSlice*volume+framesPerSlice,:,:,slice] = vol[numFramesPerVolume*volume+(slice*framesPerSlice):numFramesPerVolume*volume+(slice+1)*framesPerSlice,:,:]
    #Store the data into different slices
    for i in range(num_slice):
        sliced_data = real_data[:,:,:,0].reshape((framesPerSlice*total_volume,vol.shape[1],vol.shape[2]))
        sliced_movie = cm.movie(sliced_data)
        sliced_movie.save(cwd+'/sliced_data/'+filename[0:-4]+'_'+str(i+1)+'.tif')
    print(filename)
    print('The acutual number of frames per slice is ' + str(framesPerSlice))
    print('The acutual number of slice is ' + str(num_slice))
    print('The acutual number of volumes is ' + str(total_volume))
    print('The total frames is ' + str(total_volume*framesPerSlice*num_slice))
    print('The number of frames per volume is ' + str(numFramesPerVolume))
    print('')

The acutual number of frames per slice is 300
The acutual number of slice is 10
The acutual number of volumes is 1
The total frames is 3000
The number of frames per volume is 3000

The acutual number of frames per slice is 1
The acutual number of slice is 10
The acutual number of volumes is 1
The total frames is 10
The number of frames per volume is 10

The acutual number of frames per slice is 1
The acutual number of slice is 10
The acutual number of volumes is 1
The total frames is 10
The number of frames per volume is 10

The acutual number of frames per slice is 1
The acutual number of slice is 3
The acutual number of volumes is 200
The total frames is 600
The number of frames per volume is 3

The acutual number of frames per slice is 10
The acutual number of slice is 3
The acutual number of volumes is 10
The total frames is 300
The number of frames per volume is 30

The acutual number of frames per slice is 15
The acutual number of slice is 2
The acutual number of volumes is 100
T